<div align=center>

# Principles of Simulation: Assignment 2

By Hamed Araab & Shahriar Khalvati

</div>


### Prerequisites

In this section, we import necessary libraries and modules required for the
execution of subsequent code cells:


In [25]:
import random
import pandas as pd
import seaborn as sns

from framework import *
from __future__ import annotations

### Problem 1


### Problem 2


#### Customer


In [26]:
class Customer:
    def __init__(self) -> None:
        self.systemArrival: float
        self.server: Server | None = None
        self.serverArrival: float | None = None
        self.serviceTime: float | None = None
        self.departure: float

    @property
    def didReturn(self) -> bool:
        return (
            self.server == None
            and self.serverArrival == None
            and self.serviceTime == None
        )

    @property
    def didWaitOutside(self) -> bool:
        return self.waitingTime["outside"] not in [None, 0]

    @property
    def waitingTime(self) -> Dict[Literal["outside", "inside"], None | float]:
        return {
            "outside": (
                None if self.didReturn else self.serverArrival - self.systemArrival
            ),
            "inside": (
                None
                if self.didReturn
                else self.departure - self.serviceTime - self.serverArrival
            ),
        }

#### Server


In [27]:
class Server:
    def __init__(self, id: int, controller: ProblemController) -> None:
        self.id = id
        self.controller = controller
        self.status: Literal["available", "busy"] = "available"
        self.queue: List[Customer] = []

    @property
    def customersServed(self) -> List[Customer]:
        return [
            customer
            for customer in self.controller.customersServed
            if customer.server == self
        ]

    @property
    def utilizationPercentage(self) -> float:
        return (
            sum(customer.serviceTime for customer in self.customersServed)
            / self.controller.clock
        )

    @property
    def totalCustomersServed(self) -> int:
        return len(self.customersServed)

    @property
    def averageServiceTime(self) -> float:
        return sum(customer.serviceTime for customer in self.customersServed) / len(
            self.customersServed
        )

    @property
    def averageWaitingTime(self) -> float:
        return sum(
            customer.waitingTime["inside"] for customer in self.customersServed
        ) / len(self.customersServed)

#### Controller


In [28]:
class ProblemController(SimController):
    def __init__(self, waitOutside: bool) -> None:
        super().__init__(stopTime=3 * 60, initialEvent=ArrivalEvent(initial=True))
        self.totalCustomersArrived: int = 0
        self.customersServed: List[Customer] = []
        self.outsideQueue: List[Customer] = []
        self.servers: List[Server] = [Server(i, controller=self) for i in range(1, 4)]
        self.waitOutside = waitOutside

    def simulate(self) -> Dict[str, float]:
        super().simulate()

        customersWaitedOutside = [
            customer for customer in self.customersServed if customer.didWaitOutside
        ]

        customersNotReturned = [
            customer for customer in self.customersServed if not customer.didReturn
        ]

        results = {
            "TCA": self.totalCustomersArrived,
            f"TCWO": len(customersWaitedOutside),
            f"AWTO": (
                sum(
                    customer.waitingTime["outside"] for customer in customersNotReturned
                )
                / len(customersNotReturned)
                if customersNotReturned
                else 0
            ),
        }

        for server in self.servers:
            results |= {
                f"UP{server.id}": server.utilizationPercentage,
                f"TCS{server.id}": server.totalCustomersServed,
                f"AST{server.id}": server.averageServiceTime,
                f"AWTI{server.id}": server.averageWaitingTime,
            }

        return results

#### Event


##### Arrival


In [29]:
ARRIVAL_EVENT_INTERVAL: Callable[[], float] | None = None


class ArrivalEvent(SimEvent[ProblemController]):
    def __init__(self, initial: bool = False) -> None:
        super().__init__(0 if initial else ARRIVAL_EVENT_INTERVAL())

        self.customer = Customer()

    def trigger(self) -> None:
        self.controller.dispatchEvent(ArrivalEvent())

        self.customer.systemArrival = self.dueTime
        self.controller.totalCustomersArrived += 1

        if self.controller.outsideQueue:
            if self.controller.waitOutside:
                self.controller.outsideQueue.append(self.customer)
            else:
                self.customer.departure = self.dueTime
        else:
            server = min(self.controller.servers, key=lambda server: len(server.queue))

            if len(server.queue) + int(server.status == "busy") < 4:
                self.customer.server = server
                self.customer.serverArrival = self.dueTime

                if server.status == "available":
                    server.status = "busy"

                    self.controller.dispatchEvent(DepartureEvent(self.customer))
                else:
                    server.queue.append(self.customer)
            elif self.controller.waitOutside:
                self.controller.outsideQueue.append(self.customer)
            else:
                self.customer.departure = self.dueTime

##### Departure


In [30]:
DEPARTURE_EVENT_INTERVAL: Callable[[], float] | None = None


class DepartureEvent(SimEvent[ProblemController]):
    def __init__(self, customer: Customer) -> None:
        super().__init__(DEPARTURE_EVENT_INTERVAL())

        self.customer = customer

    def trigger(self) -> None:
        self.customer.serviceTime = self.interval
        self.customer.departure = self.dueTime

        self.controller.customersServed.append(self.customer)

        if self.customer.server.queue:
            customer = self.customer.server.queue.pop(0)

            self.controller.dispatchEvent(DepartureEvent(customer))

            if self.controller.outsideQueue:
                customer = self.controller.outsideQueue.pop(0)
                customer.server = self.customer.server
                customer.serverArrival = self.dueTime

                self.customer.server.queue.append(customer)
        else:
            self.customer.server.status = "available"

#### Results


In [31]:
ARRIVAL_EVENT_INTERVAL = lambda: DistributionFunction.uniform(0, 2)
DEPARTURE_EVENT_INTERVAL = lambda: DistributionFunction.uniform(2, 3)

allResults: List[Dict[str, float]] = []

for i in range(1000):
    results = ProblemController(waitOutside=True).simulate()

    allResults.append(results)

pd.DataFrame(allResults).mean(axis=0)

TCA      180.494000
TCWO       0.111000
AWTO       0.000415
UP1        0.995250
TCS1      71.497000
AST1       2.500402
AWTI1      2.133979
UP2        0.934954
TCS2      67.187000
AST2       2.499520
AWTI2      1.469316
UP3        0.524374
TCS3      37.687000
AST3       2.499155
AWTI3      0.969162
dtype: float64

In [32]:
ARRIVAL_EVENT_INTERVAL = lambda: DistributionFunction.uniform(0, 2)
DEPARTURE_EVENT_INTERVAL = lambda: DistributionFunction.uniform(2, 3)

allResults: List[Dict[str, float]] = []

for i in range(1000):
    results = ProblemController(waitOutside=False).simulate()

    allResults.append(results)

pd.DataFrame(allResults).mean(axis=0)

TCA      180.785000
TCWO       0.000000
AWTO       0.000000
UP1        0.994989
TCS1      71.529000
AST1       2.498625
AWTI1      2.119386
UP2        0.934079
TCS2      67.050000
AST2       2.502262
AWTI2      1.454496
UP3        0.529627
TCS3      37.981000
AST3       2.504572
AWTI3      0.958980
dtype: float64

### Problem 3
